In [36]:
import { display } from "tslab";
import { readFileSync } from "fs";

try {
    const css = readFileSync("../style.css", "utf8");
    display.html(`<style>${css}</style>`);
} catch (e) { /* ignore */ }

# A Parser for Regular Expression

This notebook implements a parser for regular expressions. The parser that is implemented in the function `parseExpr` parses a regular expression according to the following <em style="color:blue">EBNF grammar</em>.

```
regExp → product ('+' product)*
product → factor factor*
factor → atom '*'?
atom → '(' expr ')' | LETTER | '𝜀' | '∅'
```


The parse tree is represented as a nested tuple (or array in TypeScript).
- letters are represented by themselves,
- The character `'∅'` is interpreted as the regular expression $\emptyset$ denoting the empty set,
- The character `'𝜀'` represents the regular expression $\varepsilon$ denoting the empty string,
- $r_1 r_2$ is represented as `[`$r_1$ `, '⋅',` $r_2$ `]`, 
- $r_1 + r_2$ is represented as `[`$r_1$ `, '+',` $r_2$ `]`,
- $r^*$ is represented as `[` $r$ `, '*']`.

The parser is implemented as a recursive *top-down* parser.

We start with a definition of the type of the parse trees that are generated. A parse tree is either
* an integer,
* a string,
* a tuple (array) of parse trees.

We reuse the `RegExp` type defined in our `03-RegExp-2-NFA` module.

In [38]:
import { RegExp, UnaryOp, BinaryOp } from "./03-RegExp-2-NFA";

In [24]:
type ParseTree = RegExp;
type ParseResult = [ParseTree, string[]];

In [25]:
function error(msg: string, tokens: string[]): never {
    throw new Error(`Parse Error: ${msg}. Remaining tokens: ${JSON.stringify(tokens)}`);
}

### Forward Declarations

Since our parser functions `parseRegExp`, `parseProduct`, `parseFactor`, and `parseAtom` are **mutually recursive** (they call each other in a cycle), we face a challenge in a linear execution environment like a notebook. We cannot call a function that hasn't been defined yet.

To solve this, we use **forward declarations** (stubs).
1. We declare the function variables (`let`) first.
2. We assign dummy implementations that throw errors if called prematurely.
3. In subsequent cells, we overwrite these variables with the actual parsing logic.

This allows `parseAtom` (defined later) to call `parseRegExp` (defined earlier), because at runtime, the variable `parseRegExp` will hold the correct function.


In [26]:
let parseRegExp: (tokens: string[]) => ParseResult;
let parseProduct: (tokens: string[]) => ParseResult;
let parseFactor: (tokens: string[]) => ParseResult;
let parseAtom: (tokens: string[]) => ParseResult;

In [27]:
parseRegExp = () => { throw "Not implemented"; };
parseProduct = () => { throw "Not implemented"; };
parseFactor = () => { throw "Not implemented"; };
parseAtom = () => { throw "Not implemented"; };

[Function (anonymous)]


The function `tokenize(s)` partitions the string `s` into a list of tokens.
It recognizes 
- the operator symbols `+` and `*`, 
- the parentheses `(`, `)`, 
- single upper or lower case letters, 
- the symbol `∅` that matches the empty language, 
- the symbol `ε` (epsilon) that matches the empty string.

All whitespace characters (and, indeed, all characters that could not be matched) are discarded.

In [28]:
function tokenize(s: string): string[] {
    const tokenRegex = /[+*()]|[a-zA-Z]|∅|ε|∞/g; 
    const matches = s.match(tokenRegex);
    
    return matches ? matches : [];
}

The function `parseRegExp` takes a token list `TokenList` and tries to interpret this list as a regular expression.
It returns the regular expression in the form of a nested tuple and a list of those tokens that could not be parsed.
It is implemented as a <em style="color:blue">top-down-parser.</em> 

The function `parseRegExp` implements the following grammar rule:

```
regExp → product ('+' product)*
```


In [29]:
parseRegExp = function(tokens: string[]): ParseResult {
    let [result, rest] = parseProduct(tokens);

    while (rest.length > 0 && rest[0] === '+') {
        const nextTokens = rest.slice(1);
        const [right, rightRest] = parseProduct(nextTokens);
        
        result = [result, '+', right];
        rest = rightRest;
    }

    return [result, rest];
};

[Function (anonymous)]


The function `parseProduct` implements the following grammar rule:
```
product → factor factor*
```
In our regex syntax, concatenation is implicit (e.g., `ab` means `a ⋅ b`). The parser makes this explicit.

In [30]:
parseProduct = function(tokens: string[]): ParseResult {
    let [result, rest] = parseFactor(tokens);

    const isAtomStart = (t: string) => /^[a-zA-Z(∅ε]$/.test(t);

    while (rest.length > 0 && isAtomStart(rest[0])) {
        const [right, rightRest] = parseFactor(rest);
        
        result = [result, '⋅', right];
        rest = rightRest;
    }

    return [result, rest];
};


[Function (anonymous)]


The function `parseFactor` implements the following grammar rule:
```
factor → atom '*'?
```


In [31]:
parseFactor = function(tokens: string[]): ParseResult {
    let [atom, rest] = parseAtom(tokens);

    if (rest.length > 0 && rest[0] === '*') {
        atom = [atom, '*'];
        rest = rest.slice(1);
    }

    return [atom, rest];
};


[Function (anonymous)]


The function `parseAtom` implements the following grammar rule:
```
atom → '(' expr ')'
| '∅' # denotes empty set ∅
| 'ε' # denotes empty string ε
| LETTER # any letter denotes itself
```

In [32]:
parseAtom = function(tokens: string[]): ParseResult {
    if (tokens.length === 0) {
        error("Unexpected end of input", tokens);
    }

    const t = tokens[0];
    const rest = tokens.slice(1);

    if (t === '(') {
        // Hier greift er auf die Variable parseRegExp zu, die oben definiert wurde!
        const [expr, afterExpr] = parseRegExp(rest);
        
        if (afterExpr.length === 0 || afterExpr[0] !== ')') {
            error("Expected ')'", afterExpr);
        }
        
        return [expr, afterExpr.slice(1)];
    }

    if (t === '∅') return [0, rest];
    if (t === 'ε') return ['ε', rest];
    if (/^[a-zA-Z]$/.test(t)) return [t, rest];

    error(`Unexpected token '${t}'`, tokens);
    return [0, []];
};


[Function (anonymous)]


The function `parse` takes a string `s` and tries to parse it as a regular expression. It returns the parse tree.


In [34]:
function parse(s: string): ParseTree {
    const tokens = tokenize(s);
    const [result, rest] = parseRegExp(tokens);
    
    if (rest.length > 0) {
        error("Unexpected trailing tokens", rest);
    }
    
    return result;
}

In [39]:
// Test cases
console.log(parse('a*'));
console.log(parse('ab'));
console.log(parse('abc'));
console.log(parse('a+b+c'));
console.log(parse('ab+cd*'));
console.log(parse('(a+b*)*'));


[ 'a', '*' ]
[ 'a', '⋅', 'b' ]
[ [ 'a', '⋅', 'b' ], '⋅', 'c' ]
[ [ 'a', '+', 'b' ], '+', 'c' ]
[ [ 'a', '⋅', 'b' ], '+', [ 'c', '⋅', [ 'd', '*' ] ] ]
[ [ 'a', '+', [ 'b', '*' ] ], '*' ]
